In [1]:
import os,sys
import numpy as np
from collections import namedtuple
import tqdm
import glob
import math
import random
import inspect
import os.path as osp
from pathlib import Path
import itertools
from itertools import chain
import numpy as np
import pandas as pd
import multiprocessing
import h5py
import matplotlib.pyplot as plt
import numpy as np
import sys, os
from importlib import reload

import torch
import torch.nn as nn
from torch.utils.data import random_split
from torch_geometric.data import Data, DataLoader, DataListLoader
from torch_geometric.nn import EdgeConv, global_mean_pool, DataParallel
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import Data,Dataset
from torch_scatter import scatter_mean, scatter
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_geometric.nn import MetaLayer, EdgeConv, global_mean_pool, DynamicEdgeConv
import h5py

In [9]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../')))

import models_torch.models as models
reload(models)
import models_torch.losses as losses
reload(losses)
import utils_torch.scaler
reload(utils_torch.scaler)
import utils_torch.preprocessing as prepr
import utils_torch.plot_util as plot
reload(plot)
import utils_torch.train_util as train
reload(train)
import graph_data.graph_data as graph_data
reload(graph_data)
sys.path.append(os.path.abspath(os.path.join('../../')))
import vande.util.util_plotting as vande_plot
reload(vande_plot)

<module 'vande.util.util_plotting' from '/eos/home-n/nchernya/MLHEP/AnomalyDetection/vande/util/util_plotting.py'>

In [3]:
data_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/'
dataset = graph_data.GraphDataset(root=data_dir,n_jets=1e3)
save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats/'
   
use_generator = False
if use_generator:
    validation_split = 0.2
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    if dataset_size > 2:
        split = int(np.floor(validation_split * dataset_size))
    else: 
        split = 1
    print(dataset_size,split)
    random_seed= 1001

    train_subset, val_subset = torch.utils.data.random_split(dataset, [dataset_size - split, split],
                                                             generator=torch.Generator().manual_seed(random_seed))
    print("train subset dim:", len(train_subset))
    print("validation subset dim", len(val_subset))
    dataloaders = {
        'train':  DataLoader(train_subset, batch_size=128, shuffle=True),
        'val':   DataLoader(val_subset, batch_size=128, shuffle=True)
    }
    print("train_dataloader dim:", len(dataloaders['train']))
    print("val dataloader dim:", len(dataloaders['val']))
else : 
    in_memory_datas = dataset.return_inmemory_data() 

In [ ]:
pf_cands,jet_prop = dataset.pf_cands,dataset.jet_prop
pf_cands_t = torch.cat([torch.tensor(pf_cands[i], dtype=torch.float) for i in range(len(pf_cands))])
#Plot consistuents and jet features prepared for the graph! (but before any normalization)
pf_feats = 'px,py,pz,E,pt,eta,phi'.split(',')
jet_feats = 'N_constituents,M,Pt,Eta,Phi'.split(',')


save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats/'
vande_plot.plot_features(pf_cands_t.numpy(), pf_feats ,'Normalized' , 'QCD', plotname='{}plot_pf_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)
vande_plot.plot_features(jet_prop[:,0:-1], jet_feats ,'Normalized' , 'QCD', plotname='{}plot_jet_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)


In [4]:
#scaler = standardize(train_subset) # I dont think that this works for the dataset implementation as it is done now, implement scaler as part of dataset ? 
scaler = prepr.standardize(in_memory_datas,minmax_idx=[3,4],log_idx=[3,4]) 

dataloaders = {
    'train':  DataLoader(in_memory_datas, batch_size=128, shuffle=True)
    }
print("train_dataloader dim:", len(dataloaders['train']))

#dataset.get(0).u

train_dataloader dim: 8


In [ ]:
pf_cands_norm = torch.cat([torch.tensor(in_memory_datas[i].x, dtype=torch.float) for i in range(len(in_memory_datas))])
#Plot consistuents and jet features prepared for the graph! (after normalization)
pf_feats = 'px,py,pz,E,pt,eta,phi'.split(',')
save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats_normalized/'
vande_plot.plot_features(pf_cands_norm.numpy(), pf_feats ,'Normalized' , 'QCD', plotname='{}plot_pf_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)


In [5]:
torch.manual_seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
multi_gpu = False #torch.cuda.device_count()>1

In [7]:
#loss
loss_ftn_obj = losses.LossFunction('mse_coordinates', device=device)
#loss_ftn_obj = losses.LossFunction('mse', device=device)

# model
input_dim = 7
output_dim = 4#4,7
big_dim = 32
hidden_dim = 2
model = models.EdgeNet(input_dim=input_dim,output_dim=output_dim, big_dim=big_dim, hidden_dim=hidden_dim)

optimizer = torch.optim.Adam(model.parameters(), lr = 10e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4, threshold=1e-6)

model.to(device)


EdgeNet(
  (batchnorm): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): EdgeConv(nn=Sequential(
    (0): Linear(in_features=14, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
    (5): ReLU()
  ))
  (decoder): EdgeConv(nn=Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=4, bias=True)
  ))
)

In [10]:
# Training loop
n_epochs = 80
stale_epochs = 0
loss = 999999
train_losses = []
for epoch in range(0, n_epochs):
    #loss = train(model, optimizer, loader, len(datas), 128, loss_ftn_obj)
    loss = train.train(model, optimizer, dataloaders['train'], len(dataloaders['train'].dataset), dataloaders['train'].batch_size, loss_ftn_obj)
    train_losses.append(loss)
    print('Epoch: {:02d}, Training Loss:   {:.4f}'.format(epoch, loss))


  0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.8334530:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.8334530:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.6948934:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.6948934:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.6948934:  26%|██▌       | 2/7.8125 [00:00<00:00, 17.74it/s]
train loss = 0.7692120:  26%|██▌       | 2/7.8125 [00:00<00:00, 17.74it/s]
train loss = 0.7692120:  26%|██▌       | 2/7.8125 [00:00<00:00, 17.74it/s]
train loss = 0.8018770:  26%|██▌       | 2/7.8125 [00:00<00:00, 17.74it/s]
train loss = 0.8018770:  26%|██▌       | 2/7.8125 [00:00<00:00, 17.74it/s]
train loss = 0.6796097:  26%|██▌       | 2/7.8125 [00:00<00:00, 17.74it/s]
train loss = 0.6796097:  26%|██▌       | 2/7.8125 [00:00<00:00, 17.74it/s]
train loss = 0.6796097:  64%|██████▍   | 5/7.8125 [00:00<00:00, 20.04it/s]
train loss = 0.6011668:  64%|██████▍   | 5/7.8125 [00:00<00:00, 20.04it/s]
train loss = 

Epoch: 00, Training Loss:   0.6994



train loss = 0.5485487:  38%|███▊      | 3/7.8125 [00:00<00:00, 28.74it/s]
train loss = 0.5485487:  38%|███▊      | 3/7.8125 [00:00<00:00, 28.74it/s]
train loss = 0.5485487:  77%|███████▋  | 6/7.8125 [00:00<00:00, 28.85it/s]
train loss = 0.4756831:  77%|███████▋  | 6/7.8125 [00:00<00:00, 28.85it/s]
train loss = 0.4756831:  77%|███████▋  | 6/7.8125 [00:00<00:00, 28.85it/s]
train loss = 0.4829677:  77%|███████▋  | 6/7.8125 [00:00<00:00, 28.85it/s]
train loss = 0.4829677:  77%|███████▋  | 6/7.8125 [00:00<00:00, 28.85it/s]
train loss = 0.4829677: : 8it [00:00, 29.08it/s]                          
  0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.6121086:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.6121086:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.5403536:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.5403536:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.5944874:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.594487

Epoch: 01, Training Loss:   0.5555



train loss = 0.5927446:  38%|███▊      | 3/7.8125 [00:00<00:00, 28.98it/s]
train loss = 0.5927446:  38%|███▊      | 3/7.8125 [00:00<00:00, 28.98it/s]
train loss = 0.5927446:  77%|███████▋  | 6/7.8125 [00:00<00:00, 28.91it/s]
train loss = 0.4857911:  77%|███████▋  | 6/7.8125 [00:00<00:00, 28.91it/s]
train loss = 0.4857911:  77%|███████▋  | 6/7.8125 [00:00<00:00, 28.91it/s]
train loss = 0.4330948:  77%|███████▋  | 6/7.8125 [00:00<00:00, 28.91it/s]
train loss = 0.4330948:  77%|███████▋  | 6/7.8125 [00:00<00:00, 28.91it/s]
train loss = 0.4330948: : 8it [00:00, 28.85it/s]                          
  0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.5138445:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.5138445:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.3961394:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.3961394:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.4673441:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.467344

...

Epoch: 77, Training Loss:   0.1627


train loss = 0.1555887:  51%|█████     | 4/7.8125 [00:00<00:00, 30.21it/s]
train loss = 0.1555887:  51%|█████     | 4/7.8125 [00:00<00:00, 30.21it/s]
train loss = 0.1332035:  51%|█████     | 4/7.8125 [00:00<00:00, 30.21it/s]
train loss = 0.1332035:  51%|█████     | 4/7.8125 [00:00<00:00, 30.21it/s]
train loss = 0.1332035:  90%|████████▉ | 7/7.8125 [00:00<00:00, 30.05it/s]
train loss = 0.1473888:  90%|████████▉ | 7/7.8125 [00:00<00:00, 30.05it/s]
train loss = 0.1473888:  90%|████████▉ | 7/7.8125 [00:00<00:00, 30.05it/s]
train loss = 0.1473888: : 8it [00:00, 30.18it/s]                          
  0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.1485485:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.1485485:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.1278569:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.1278569:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.1940827:   0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 0.1940827

Epoch: 78, Training Loss:   0.1582



train loss = 0.1779136:  51%|█████     | 4/7.8125 [00:00<00:00, 30.35it/s]
train loss = 0.1744689:  51%|█████     | 4/7.8125 [00:00<00:00, 30.35it/s]
train loss = 0.1744689:  51%|█████     | 4/7.8125 [00:00<00:00, 30.35it/s]
train loss = 0.1744689:  90%|████████▉ | 7/7.8125 [00:00<00:00, 30.08it/s]
train loss = 0.1349699:  90%|████████▉ | 7/7.8125 [00:00<00:00, 30.08it/s]
train loss = 0.1349699:  90%|████████▉ | 7/7.8125 [00:00<00:00, 30.08it/s]
train loss = 0.1349699: : 8it [00:00, 30.13it/s]                          

Epoch: 79, Training Loss:   0.1623


In [16]:
reload(utils_torch.scaler)
reload(prepr)
reload(plot)

inverse_standardization = False
plot.plot_reco_for_loader(model, dataloaders['train'], device, scaler, inverse_standardization, 'test_train', osp.join(save_dir, 'reconstruction_post_train', 'train_reco_std_log_minmax_mseconv'), 'all_norm_mseconv')


In [88]:
scaler_new = utils_torch.scaler.Standardizer(minmax_idx=[3,4],  log_idx=[3,4])
scaler_new.std=scaler.std
scaler_new.mean=scaler.mean
scaler_new.max=scaler.max
scaler_new.min=scaler.min
scaler_new.minmax_idx = scaler.minmax_idx
scaler_new.std_idx = scaler.std_idx
scaler_new.log_idx = scaler.log_idx